# JSON Cue & Lick Explorer

Inspect cue windows, licking behaviour, and trial outcomes for DELTA sessions without
wiring every processing step inside the notebook.

## Workflow
1. Configure which JSON log to analyse and the cue window size.
2. Run the loader cell to build session artefacts using the processing pipeline.
3. Review trial/lick summaries and use the plots to inspect behaviour across time or corridors.

In [ ]:
from pathlib import Path
import sys

import pandas as pd

# Make sure the repository src/ directory is importable whether the kernel started in the repo
NOTEBOOK_DIR = Path.cwd().resolve()
if (NOTEBOOK_DIR / "src").exists():
    REPO_ROOT = NOTEBOOK_DIR
else:
    REPO_ROOT = NOTEBOOK_DIR.parent

SRC_PATH = REPO_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

from behavioral_analysis.visualization.cue_lick_explorer import (
    SessionArtifacts,
    load_session_artifacts,
    build_trial_summary,
    build_lick_summary,
    plot_trial_overview,
    plot_lick_offset_histogram,
)


In [ ]:
# --- Session configuration ------------------------------------------------------------
JSON_PATH = (REPO_ROOT / "Log BM35 2025-09-22 session 1.json").resolve()
CORRIDOR_LENGTH_CM = 500.0
CUE_WINDOW_HALF_WIDTH_CM = 10.0
TIMELINE_X_AXIS = "time"  # "time" or "corridor"
VERBOSE = False

available_logs = sorted(REPO_ROOT.glob("Log *.json"))
print(f"Found {len(available_logs)} JSON log(s) under {REPO_ROOT}")
for path in available_logs[:8]:
    print(" -", path.relative_to(REPO_ROOT))
if len(available_logs) > 8:
    print("   … (use available_logs to inspect the rest)")

JSON_PATH

In [ ]:
session = load_session_artifacts(
    JSON_PATH,
    corridor_length_cm=CORRIDOR_LENGTH_CM,
    cue_window_half_width_cm=CUE_WINDOW_HALF_WIDTH_CM,
    verbose=VERBOSE,
)

print(f"Corridors detected: {session.corridor_info['corridor_id'].nunique() if not session.corridor_info.empty else 0}")
print(f"Trials created: {len(session.trials)}")

print(f"Lick events with position: {len(session.alignment.licks_with_position)}")
print(f"Licks inside cue windows: {len(session.alignment.licks_with_trial)}")

display(session.event_counts.to_frame(name="count"))


In [ ]:
pd.set_option("display.max_rows", 20)
display(session.trials.head())

trial_summary = build_trial_summary(session.trials)
if trial_summary.empty:
    print("No trial data available.")
else:
    display(trial_summary)

In [ ]:
alignment = session.alignment

print(f"Cue window half-width: ±{session.cue_window_half_width_cm:g} cm")
display(alignment.trials.head())


In [ ]:
lick_summary = build_lick_summary(session.alignment.licks_with_trial)
if lick_summary.empty:
    print("No licks detected inside cue windows.")
else:
    display(lick_summary)


In [ ]:
ax = plot_trial_overview(
    session.trials,
    session.alignment.licks_with_position,
    cue_window_half_width_cm=session.cue_window_half_width_cm,
    x_axis=TIMELINE_X_AXIS,
)
ax.figure


In [ ]:
ax = plot_lick_offset_histogram(session.alignment.licks_with_trial)
ax.figure
